In [1]:
# Library Imports 

import pandas as pd # for dataframe 
import seaborn as sns # for seaborn visualizattion
import matplotlib.pyplot as plt # for matplot visualization
import numpy as np # for numerical computing
from sklearn.preprocessing import StandardScaler # for data normalization
from sklearn.model_selection import train_test_split # for data split


**Feature set description**

|**Feature**         |**Description**|
|----------------|:----------------|
|Date Time       | year-month-day hour:minute:second   |
|Appliances      | energy use in Wh|
|lights          | energy use of light fixtures in the house in Wh|
|T1              | Temperature in kitchen area, in Celsius|
|RH_1            | Temperature in kitchen area, in Celsius|
|T2              | Temperature in living room area, in Celsius
|RH_2            | Humidity in living room area, in %
|T3              | Temperature in laundry room area
|RH_3            | Humidity in laundry room area, in %
|T4              | Temperature in office room, in Celsius
|RH_4            | Humidity in office room, in %
|T5              | Temperature in bathroom, in Celsius
|RH_5            | Humidity in bathroom, in %
|T6              | Temperature outside the building (north side), in Celsius
|RH_6            | Humidity outside the building (north side), in %
|T7              | Temperature in ironing room , in Celsius
|RH_7            | Humidity in ironing room, in %
|T8              | Temperature in teenager room 2, in Celsius
|RH_8            | Humidity in teenager room 2, in %
|T9              | Temperature in parents room, in Celsius
|RH_9            | Humidity in parents room, in %
|To              | Temperature outside (from Chievres weather station), in Celsius
|Pressure (from Chievres weather station) | in mm Hg
|RH_out Humidity outside (from Chievres weather station) | in %
|Wind speed (from Chievres weather station) | in m/s
|Visibility (from Chievres weather station) | in km
|Tdewpoint (from Chievres weather station) | Â°C
|rv1 | Random variable 1 nondimensional
|rv2 | Random variable 2 nondimensional


In [ ]:
file_path = 'energydata_complete.csv'

# Read our dataset into our dataframe
df = pd.read_csv(file_path)

# Print the head of the dataframe
df.head()

In [ ]:
# Check for missing values in the features
df.isna().sum()


Per our output, we see not missing values in the dataset. 

In [4]:
# Conver date to Datetime format
df['Datetime'] = pd.to_datetime(df['date'])

In [ ]:
# Verify our dataframe types 
df.dtypes

In [6]:
# Round all values to 2 decimal places for easier processing and formatting
df = df.round(2)

In [ ]:
# Validate our rounding
df.head()

In [8]:
# Drop the 'Date' column
df = df.drop(columns=['date'])

# Move the 'Datetime' column to the first position
df = df[['Datetime'] + [col for col in df.columns if col != 'Datetime']]

In [ ]:
df.head()

In [10]:
correlation_matrix = df.corr()

In [ ]:
# Set up the figure size
plt.figure(figsize=(12, 10))

# Plot the heatmap
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f", cbar=True)

# Add titles and labels
plt.title('Heatmap of Correlations Between Variables', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

# Display the heatmap
plt.show()

The below features were chosen based on their strong correlation with Appliances (energy consumption) while avoiding redundant or weak predictors. <br>
lights was included because lighting directly contributes to household energy use. Indoor temperature features (T1, T2) were selected as they <br>
impact HVAC operations, which are a major factor in energy consumption. T_out (outdoor temperature) and Tdewpoint were chosen because external <br>
weather conditions influence heating and cooling requirements inside the home. RH_out (outdoor humidity) and Windspeed were kept as they can affect <br>
temperature regulation and ventilation needs. Finally, hour was included to capture daily energy usage patterns, as appliance consumption tends to <br>
vary throughout the day. These features provide a balanced mix of internal conditions, external environmental influences, and time-based patterns, <br>
ensuring the model captures key factors affecting energy consumption.

|**Feature**         |**Description**|
|----------------|:----------------|
|lights                    | Directly affects energyh consumption  |
|T1 (Kitchen Temp)         | Strong correlation with Appliances    |
|T2 (Living Room Temp)     | Represents HVAC energy use            |
|T_out (Outside Temp)      | External factor on energy consumption |
|RH_out (Outdoor Humidity) | Impcasts cooling and heating          | 
|Windspeed                 | Influences temp reulation             | 
|Tdewpoint                 | Outdoor conditions                    | 
|hour                      | Captures time-based energy patterns   | 

In [ ]:
# Plot a distribution of the energy consumption
sns.histplot(df['Appliances'], bins=30, kde=True)
plt.title("Appliances Energy Consumption Distribution")
plt.xlabel("Energy Consumption (Wh)")
plt.ylabel("Occurrence")
plt.show()


1. We can see from our distribution plot for energy consumption that most of the consumption is < 200 Wh
2. A sharp peak > 50 Wh < 100 Wh suggests many appliances consume low energy. 
3. We have a right skewed plot, dmeonstrating that anything > 400 Wh is rare. 

In [13]:
# Extract time-based features
df['hour'] = df['Datetime'].dt.hour  # Extract hour of the day (0-23)


In [ ]:
# Plot raw data (individual appliance consumption points)
plt.figure(figsize=(10, 5))
plt.scatter(df['hour'], df['Appliances'], alpha=0.3, color='blue', label="Appliance Energy Consumption")

# Overlay the 24-hour average trend
hourly_avg = df.groupby('hour')['Appliances'].mean()
plt.plot(hourly_avg.index, hourly_avg.values, marker='o', linestyle='-', color='red', label="Hourly Average")

# Plot our hourly average across the dataset
plt.xlabel("Hour of the Day")
plt.ylabel("Energy Consumption (Wh)")
plt.title("Appliance Energy Consumption by Hour of the Day")
plt.legend()
plt.grid()
plt.show()

The hourly average did not provide to strong of an insight. As expected, energy consumption is lowest during sleeping hours. <br>
Early evening has the peak on average. 

In [15]:
# Keep only the features we want in our dataframe
df = df[['Appliances', 'lights', 'T1', 'T2', 'T_out', 'RH_out', 'Windspeed', 'Tdewpoint', 'hour']]

In [16]:
# Prepare our data for training
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop(columns=['Appliances']))
y = df['Appliances']

In [17]:
# Splitting our dataset into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)